In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [2]:
tf.enable_eager_execution()
num_epochs = 1
batch_size = 32
learning_rate = 0.001

In [3]:
data_dir = './data'
train_bottle_dir = data_dir + '/train/塑料瓶/'
train_chopsticks_dir = data_dir + '/train/一次性筷子/'

In [4]:
def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)  # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [256, 256]) / 255.0
    return image_resized, label

In [5]:
train_bottle_filenames = tf.constant([train_bottle_dir + filename for filename in os.listdir(train_bottle_dir)])
train_chopsticks_filenames = tf.constant(
    [train_chopsticks_dir + filename for filename in os.listdir(train_chopsticks_dir)])
train_filenames = tf.concat([train_bottle_filenames, train_chopsticks_filenames], axis=-1)
train_labels = tf.concat([
    tf.zeros(train_bottle_filenames.shape, dtype=tf.int32),
    tf.ones(train_chopsticks_filenames.shape, dtype=tf.int32)],
    axis=-1)
train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
train_dataset = train_dataset.map(
    map_func=_decode_and_resize,
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [13]:
train_dataset = train_dataset.shuffle(buffer_size=100)    
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [15]:
model = tf.keras.applications.ResNet50(weights=None,classes=2)
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )

model.fit(train_dataset,epochs=num_epochs,steps_per_epoch = 10)

InvalidArgumentError: Got 17 frames, but animated gifs can only be decoded by tf.image.decode_gif or tf.image.decode_image
	 [[{{node DecodeJpeg}}]] [Op:IteratorGetNextSync]